# Importing Libraries

In [15]:
# Importing the import-ant libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score
import tensorflow as tf
import csv
import numpy as np
from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials


# Dealing With Spreadsheets

In [16]:
# Auth to allow access to the spread sheet
auth.authenticate_user()

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [32]:
# Opening the spreadsheet and downloading as CSV
worksheet = gc.open("Spoof Data").worksheet('Sheet5')

rows = worksheet.get_all_values()

pd.DataFrame.from_records(rows).to_csv("data.csv",index=False,header=False)

# Preprocessing Data

In [33]:
# Reading the CSV
df = pd.read_csv("data.csv")

In [34]:
# Preprocessing data
X = pd.get_dummies(df.drop(["Letter"],axis=1)) # Remove letter as that outcome

# List of the alphabets in order 👍🏻
# alphabet = ["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q",
#            "R","S","T","U","V","W","X","Y","Z"]
letters = ["A","B","C","D","E","F","G","H","I","K","L","M","N","O","P","Q",
            "S","T","U","W","X","Y"]

Y = df["Letter"].apply(lambda x: letters.index(x)) # Mapping ints to the letters
print(Y)
print(X)

0        0
1        0
2        0
3        0
4        0
        ..
1095    21
1096    21
1097    21
1098    21
1099    21
Name: Letter, Length: 1100, dtype: int64
         Pinky      Ring    Middle     Index     Thumb
0     0.899785  0.864742  0.845010  0.960488  0.195136
1     0.961064  0.990950  0.852364  0.822135  0.156835
2     0.982067  0.813264  0.867312  0.857287  0.171330
3     0.828814  0.928809  0.935763  0.978742  0.246543
4     0.897464  0.901527  0.896102  0.880796  0.180406
...        ...       ...       ...       ...       ...
1095  0.271616  0.601854  0.960353  0.964851  0.157433
1096  0.306891  0.615226  0.950689  0.970468  0.195694
1097  0.141996  0.604050  0.980890  0.948656  0.060170
1098  0.295461  0.580406  0.942298  0.925024  0.201908
1099  0.235419  0.526062  0.928020  0.915859  0.199592

[1100 rows x 5 columns]


# Getting Model

In [35]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y, test_size=0.3)

In [36]:
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Dense(5, activation="relu"),
                                    tf.keras.layers.Dense(64, activation="relu"),
                                    tf.keras.layers.Dense(32, activation="relu"),
                                    tf.keras.layers.Dense(len(letters), activation="softmax")
])

In [37]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


# Training Model

In [38]:
model.fit(X_train, Y_train, epochs=200)

Epoch 1/200
25/25 [==============================] - 1s 2ms/step - loss: 3.0817 - accuracy: 0.0429
Epoch 2/200
25/25 [==============================] - 0s 2ms/step - loss: 3.0534 - accuracy: 0.1182
Epoch 3/200
25/25 [==============================] - 0s 2ms/step - loss: 3.0105 - accuracy: 0.1351
Epoch 4/200
25/25 [==============================] - 0s 2ms/step - loss: 2.9364 - accuracy: 0.1338
Epoch 5/200
25/25 [==============================] - 0s 2ms/step - loss: 2.8147 - accuracy: 0.1610
Epoch 6/200
25/25 [==============================] - 0s 2ms/step - loss: 2.6380 - accuracy: 0.1987
Epoch 7/200
25/25 [==============================] - 0s 2ms/step - loss: 2.4452 - accuracy: 0.2286
Epoch 8/200
25/25 [==============================] - 0s 2ms/step - loss: 2.2593 - accuracy: 0.2688
Epoch 9/200
25/25 [==============================] - 0s 2ms/step - loss: 2.0902 - accuracy: 0.2766
Epoch 10/200
25/25 [==============================] - 0s 2ms/step - loss: 1.9399 - accuracy: 0.3714
Epoch 11/

# Testing Model

In [39]:
model.evaluate(X_test, Y_test)

11/11 [==============================] - 0s 2ms/step - loss: 0.3327 - accuracy: 0.8424


[0.3327272832393646, 0.842424213886261]

# Saving Model


In [47]:
model.save_weights('./weights/weights1')

# Load and Test Model

In [41]:
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Dense(5, activation="relu"),
                                    tf.keras.layers.Dense(64, activation="relu"),
                                    tf.keras.layers.Dense(32, activation="relu"),
                                    tf.keras.layers.Dense(len(letters), activation="softmax")
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.load_weights('./weights/weights1')


# Predicting the Model


In [46]:
pWork = gc.open("Spoof Data").worksheet('Sheet3')

pRows = pWork.get_all_values()

pd.DataFrame.from_records(pRows).to_csv("pData.csv",index=False,header=False)

pDf = pd.read_csv("pData.csv")

pX = pd.get_dummies(pDf.drop(["Letter"],axis=1))

predictions = model.predict(pX)

print(predictions)

classes = np.argmax(predictions, axis = 1)
print(classes)

letters = ["A","B","C","D","E","F","G","H","I","K","L","M","N","O","P","Q",
           "S","T","U","W","X","Y"]
print(letters[classes[0]])



[[2.7369201e-11 0.0000000e+00 2.9566844e-24 3.3884774e-06 6.5052062e-20
  0.0000000e+00 1.6102745e-01 1.0203554e-21 0.0000000e+00 2.7549616e-12
  8.3883429e-01 1.5755538e-21 5.4195311e-26 0.0000000e+00 6.8864603e-05
  1.9569273e-09 6.0251686e-37 6.6029614e-05 1.2015676e-19 0.0000000e+00
  1.9076561e-22 5.5424499e-17]]
[10]
L
